# 1 - Statistiques descriptives

## Importation des modules

In [ ]:
# Rechargement des imports
%load_ext autoreload
%autoreload 2

# Importation des modules
# Modules de base
import numpy as np
import pandas as pd
from tqdm import tqdm
import yaml
import sys

sys.path.append("..")

# Importation du module du package
from bozio_wasmer_simulations.description.base import DescriptionBuilder

# Chargement du fichier de configurations
with open("../config.yaml") as file:
    config = yaml.safe_load(file)

# Création des scénarios à décrire
scenarios = {
    k: config["SCENARIOS"][k]
    for k in config["SCENARIOS"]
    if k in config["DESCRIPTION"]["SCENARIOS"]
}

## Importation et preprocessing des données

In [ ]:
# Initialisation d'une instance de la classe
description = DescriptionBuilder(project=config['CASD']['PROJET'])
# Construction des données
description.build(
    year=config["SIMULATIONS"]["CORE"]["YEAR_SIMUL"],
    scenarios=scenarios,
    name_firm_elasticite="firm_0.2",
    elasticite_names=config["DESCRIPTION"]["ELASTICITES"],
)
# Exportation
description.data_simulation.to_csv(
    f"../data/data_simulation_{config['SIMULATIONS']['CORE']['YEAR_SIMUL']}_emploi.csv",
    index=False,
)

description.data_simulation.head()

## Statistiques descriptives

### Itération de statistiques descriptives

In [ ]:
# Liste des statistiques descriptives à itérer
list_dict_stat_des = np.concatenate(
    [
        [
            ## Statistiques descriptives sur le coût du travail
            # Répartition des individus par tranche d'évolution du coût du travail et de la masse salariale
            {
                "list_var_groupby": [f"tranche_evol_ct_{name.lower()}"],
                "list_var_of_interest": ["quotite_de_travail", "salaire_super_brut"],
                "output_folder_path": name.lower(),
                "export_filename": "tranche_evol_ct",
            },
            {
                "list_var_groupby": [f"tranche_evol_ms_{name.lower()}"],
                "list_var_of_interest": ["quotite_de_travail", "salaire_super_brut"],
                "output_folder_path": name.lower(),
                "export_filename": "tranche_evol_ms",
            },
            # Caractérisation des évolutions de coût du travail
            {
                "list_var_groupby": ["categorie_entreprise"],
                "list_var_of_interest": [
                    f"diff_ct_{name.lower()}",
                    "salaire_super_brut",
                ],
                "output_folder_path": name.lower(),
                "export_filename": "evol_ct_categorie_entreprise",
            },
            {
                "list_var_groupby": ["a17"],
                "list_var_of_interest": [
                    f"diff_ct_{name.lower()}",
                    "salaire_super_brut",
                ],
                "output_folder_path": name.lower(),
                "export_filename": "evol_ct_a17",
            },
            {
                "list_var_groupby": ["a17_interim_nettoyage"],
                "list_var_of_interest": [
                    f"diff_ct_{name.lower()}",
                    "salaire_super_brut",
                ],
                "output_folder_path": name.lower(),
                "export_filename": "evol_ct_a17_interim_nettoyage",
            },
            # Statistiques descriptives sur les effets emploi
            {
                "list_var_groupby": ["categorie_entreprise"],
                "list_var_of_interest": ["quotite_de_travail"]
                + [
                    f"effet_emploi_{name_elast}_{name.lower()}"
                    for name_elast in config["DESCRIPTION"]["ELASTICITES_DESCRIPTION"]
                ],
                "output_folder_path": name.lower(),
                "export_filename": "evol_emploi_categorie_entreprise",
            },
            {
                "list_var_groupby": ["tranche_salaire_brut_smic"],
                "list_var_of_interest": ["quotite_de_travail"]
                + [
                    f"effet_emploi_{name_elast}_{name.lower()}"
                    for name_elast in config["DESCRIPTION"]["ELASTICITES_DESCRIPTION"]
                ],
                "output_folder_path": name.lower(),
                "export_filename": "evol_emploi_tranche_smic",
            },
            {
                "list_var_groupby": ["a17"],
                "list_var_of_interest": ["quotite_de_travail"]
                + [
                    f"effet_emploi_{name_elast}_{name.lower()}"
                    for name_elast in config["DESCRIPTION"]["ELASTICITES_DESCRIPTION"]
                ],
                "output_folder_path": name.lower(),
                "export_filename": "evol_emploi_a17",
            },
            {
                "list_var_groupby": ["a17_interim_nettoyage"],
                "list_var_of_interest": ["quotite_de_travail"]
                + [
                    f"effet_emploi_{name_elast}_{name.lower()}"
                    for name_elast in config["DESCRIPTION"]["ELASTICITES_DESCRIPTION"]
                ],
                "output_folder_path": name.lower(),
                "export_filename": "evol_emploi_a17_interim_nettoyage",
            },
        ]
        for name in config["DESCRIPTION"]["SCENARIOS"]
    ]
)

# Itération des statistiques descriptives
for dict_stat_des in tqdm(list_dict_stat_des):
    description.stat_des(
        year=config["SIMULATIONS"]["CORE"]["YEAR_SIMUL"],
        list_var_groupby=dict_stat_des["list_var_groupby"],
        list_var_of_interest=dict_stat_des["list_var_of_interest"],
        output_folder_path=dict_stat_des["output_folder_path"],
        export_filename=dict_stat_des["export_filename"],
    )

### Effets de premier tour

#### Masse salariale

In [ ]:
# Construction du jeu de données
data_1t_ms = description.first_round_effects(
    data=description.data_simulation,
    variable="salaire_brut",
    name_elasticite="indiv_0.4-0_firm_0.2",
    scenarios=config["DESCRIPTION"]["SCENARIOS"],
)
data_1t_ms.head()

#### Cotisations sociales

In [ ]:
# Construction du jeu de données
data_1t_cs = description.first_round_effects(
    data=description.data_simulation,
    variable="cotisations_sociales",
    name_elasticite="indiv_0.4-0_firm_0.2",
    scenarios=config["DESCRIPTION"]["SCENARIOS"],
)
data_1t_cs.head()